I used the average number of syllables

# IRTM Second Project

Ghadamiyan Lida, class 507

In [1]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
nltk.download('wordnet')
nltk.download('punkt')
stemmer = SnowballStemmer('english')

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler

from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.cluster import KMeans, DBSCAN

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import fowlkes_mallows_score

import seaborn as sns
import cufflinks as cf


from sklearn.pipeline import Pipeline 
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV

import pyphen

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lidag\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lidag\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data preprocessing

In [2]:
data = pd.read_csv("Lyrics-Genre-Train.csv")
data1 = pd.read_csv("Lyrics-Genre-Test-GroundTruth.csv")

In [3]:
def conv_labels(data):          # Converting the labels to numbers

    
    numeric_labels=[]

    for i in range(0, len(data.index)):

        if data.Genre[i] == 'Metal':
            numeric_labels.append(0)

        if data.Genre[i] == 'Hip-Hop':
            numeric_labels.append(1)

        if data.Genre[i] == 'Country':
            numeric_labels.append(2)

        if data.Genre[i] == 'Jazz':
            numeric_labels.append(3)

        if data.Genre[i] == 'Electronic':
            numeric_labels.append(4)

        if data.Genre[i] == 'Pop':
            numeric_labels.append(5)

        if data.Genre[i] == 'Folk':
            numeric_labels.append(6)

        if data.Genre[i] == 'Rock':
            numeric_labels.append(7)

        if data.Genre[i] == 'R&B':
            numeric_labels.append(8)

        if data.Genre[i] == 'Indie':
            numeric_labels.append(9)
            
    return numeric_labels

In [4]:
numeric_labels = conv_labels(data)
numeric_labels1 = conv_labels(data1)

In [5]:
def prep(data):                             
    
    data2 = []
    for i in range(0, len(data.index)):

        # Punctuation removal
        table = str.maketrans(dict.fromkeys(string.punctuation))                   
        sentences = (data.Lyrics[i].translate(table))

        # " '\n " removal
        words = sentences[:-3] 

        # Tokenization
        words = nltk.word_tokenize(words)

        # shrt words removal & lemmatization & stemming
        words_ = []
        for word in words:
            if len(word) > 2:   
                if not word.isnumeric():                                                  
                    word1 = stemmer.stem(WordNetLemmatizer().lemmatize(word, pos='v'))          
                    words_.append(word1)
        data2.append(words_)  
    return data2

In [6]:
df = pd.DataFrame({'lyr':prep(data), 'gen':numeric_labels})   

In [7]:
train_data__ = df['lyr']
train_labels = df['gen']

In [8]:
df_test = pd.DataFrame({'lyr':prep(data1), 'gen':numeric_labels1})   

test_data__ = df_test['lyr']
test_labels = df_test['gen']

In [9]:
# CountVectorizer & TermFrequencies
cvect = CountVectorizer(ngram_range=(1, 1), lowercase='true')   
tfidf_transformer = TfidfTransformer(norm= 'l2', use_idf= True)

In [10]:
# Transforming the processed data to a list (for tfidf)
data4 = train_data__.astype(str).values.tolist()

train_data1 = cvect.fit_transform(data4)
train_data = tfidf_transformer.fit_transform(train_data1)

In [11]:
# Same procedure for the test data
data5 = test_data__.astype(str).values.tolist()

test_data1 = cvect.transform(data5)
test_data = tfidf_transformer.transform(test_data1)

## Additional features extracted from the lyrics

In [12]:
# Extracting the number of syllables and adding in df and df_test

dic = pyphen.Pyphen(lang='en')

In [46]:
dic.inserted('hello wolrd, today is such a beautiful day')

'hel-lo -wolrd, today is such a -beau-ti-ful day'

In [15]:
def get_syllables(df):
    
    syl = []
    for i in range(0, len(df.index)):
        
        # Punctuation removal
        table = str.maketrans(dict.fromkeys(string.punctuation))                   
        sentences = (df.Lyrics[i].translate(table))

        # " '\n " removal
        words = sentences[:-3] 

        # Tokenization
        words = nltk.word_tokenize(words)
        
        s =''
        for word in words:
            s += ' '+str(dic.inserted(word))
            s = s.replace('-', ' ')
    
        syl.append(len(s)/len(df.Lyrics[i]))
        
    return syl

In [17]:
syl = get_syllables(data)

In [39]:
df_train = pd.DataFrame({'avg_syl': syl}) 

In [35]:
df_test_ = pd.DataFrame({'avg_syl': get_syllables(data1)}) 

## SVM 0.193

In [41]:
from sklearn import svm

model = svm.SVC()
model.fit(df_train, train_labels)

SVC()

In [43]:
prediction = model.predict(df_test_)

In [44]:
accuracy_score(prediction, test_labels)

0.19382482671707624

## Random Forest 0.194

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(df_train, train_labels)

RandomForestClassifier(max_depth=2, random_state=0)

In [45]:
p = clf.predict(df_test_)
accuracy_score(p, test_labels)

0.19458097038437303